In [1]:
import pandas as pd

# Load cleaned CSVs for each position
df_gk = pd.read_csv('df_gk_cleaned.csv')
df_def = pd.read_csv('df_def_cleaned.csv')
df_mid = pd.read_csv('df_mid_cleaned.csv')
df_fwd = pd.read_csv('df_fwd_cleaned.csv')

# Check the head of each dataframe to ensure the data is correctly loaded
print("GK Data:")
print(df_gk.head())

print("\nDEF Data:")
print(df_def.head())

print("\nMID Data:")
print(df_mid.head())

print("\nFWD Data:")
print(df_fwd.head())

GK Data:
             name position           team  next_GW_total_points  GW   season  \
0  aaron ramsdale       GK  Sheffield Utd                     2   1  2020-21   
1  aaron ramsdale       GK  Sheffield Utd                     4   2  2020-21   
2  aaron ramsdale       GK  Sheffield Utd                     2   3  2020-21   
3  aaron ramsdale       GK  Sheffield Utd                     3   4  2020-21   
4  aaron ramsdale       GK  Sheffield Utd                     2   5  2020-21   

    xP  bps  goals_conceded               kickoff_time  ... selected  \
0  2.5   12               2  2020-09-14 17:00:00+00:00  ...   462939   
1  2.0   10               1  2020-09-21 17:00:00+00:00  ...   440147   
2  2.8   26               1  2020-09-27 11:00:00+00:00  ...   377916   
3  1.7   13               2  2020-10-04 13:00:00+00:00  ...   335363   
4  3.3   18               1  2020-10-18 11:00:00+00:00  ...   320507   

   transfers_balance  value  was_home  expected_goals_conceded  \
0          

In [2]:
import joblib

# Load the pre-trained models for each position
gk_xgb_high = joblib.load('gk_xgb_high_model.pkl')
gk_xgb_low = joblib.load('gk_xgb_low_model.pkl')

def_xgb_high = joblib.load('def_xgb_high_model.pkl')
def_xgb_low = joblib.load('def_xgb_low_model.pkl')

mid_xgb_high = joblib.load('mid_xgb_high_model.pkl')
mid_xgb_low = joblib.load('mid_xgb_low_model.pkl')

fwd_xgb_high = joblib.load('fwd_xgb_high_model.pkl')
fwd_xgb_low = joblib.load('fwd_xgb_low_model.pkl')


In [ ]:
# Function to classify players into high and low scorers
def classify_players(position_data, threshold=6):
    """
    Classify players into high and low scorers based on their predicted points for GW 153.
    """
    position_data['Scorer_Type'] = position_data['Predicted_Points'].apply(lambda x: 'High' if x >= threshold else 'Low')
    return position_data

# Apply classification for all positions
classified_gk = classify_players(predicted_gk, threshold=6)
classified_def = classify_players(predicted_def, threshold=6)
classified_mid = classify_players(predicted_mid, threshold=6)
classified_fwd = classify_players(predicted_fwd, threshold=6)

# Combine classified players data
classified_all_positions = pd.concat([classified_gk, classified_def, classified_mid, classified_fwd])

# Check the classified data
print(classified_all_positions[['name', 'team', 'Predicted_Points', 'Scorer_Type']].head())


In [ ]:
# Prepare to predict GW 154 using the high and low scorer models
def predict_gw_154(position_data, model_high, model_low, features):
    """
    Predict the next GW total points for GW 154 using classified players from GW 153.
    """
    # Separate high and low scorers based on classification
    high_scorers = position_data[position_data['Scorer_Type'] == 'High']
    low_scorers = position_data[position_data['Scorer_Type'] == 'Low']

    # Preprocess and scale the features for both groups
    X_high = high_scorers[features]
    X_low = low_scorers[features]
    
    scaler = StandardScaler()
    X_high_scaled = scaler.fit_transform(X_high)
    X_low_scaled = scaler.fit_transform(X_low)

    # Predict for high and low scorers
    high_scorer_predictions = model_high.predict(X_high_scaled)
    low_scorer_predictions = model_low.predict(X_low_scaled)

    # Assign predictions to the dataframe for GW 154
    high_scorers['Predicted_GW_154'] = high_scorer_predictions
    low_scorers['Predicted_GW_154'] = low_scorer_predictions
    
    # Combine high and low scorers back together
    combined_predictions = pd.concat([high_scorers, low_scorers])
    return combined_predictions

# Predict for GW 154 for each position
gw_154_gk = predict_gw_154(classified_gk, gk_xgb_high, gk_xgb_low, features_gk)
gw_154_def = predict_gw_154(classified_def, def_xgb_high, def_xgb_low, features_def)
gw_154_mid = predict_gw_154(classified_mid, mid_xgb_high, mid_xgb_low, features_mid)
gw_154_fwd = predict_gw_154(classified_fwd, fwd_xgb_high, fwd_xgb_low, features_fwd)

# Combine all positions' predictions for GW 154
predicted_gw_154_all_positions = pd.concat([gw_154_gk, gw_154_def, gw_154_mid, gw_154_fwd])

# Check the predictions for GW 154
print(predicted_gw_154_all_positions[['name', 'team', 'Predicted_GW_154']].head())


In [ ]:
# Save the GW 153 predictions to a CSV file
classified_all_positions.to_csv('gw_153_predictions.csv', index=False)

# Save the GW 154 predictions to a CSV file
predicted_gw_154_all_positions.to_csv('gw_154_predictions.csv', index=False)


In [29]:
# Import necessary libraries
import pandas as pd
import sasoptpy as so
import os
import time

# Load the cleaned data for different positions (GK, DEF, MID, FWD)
df_gk = pd.read_csv('df_gk_cleaned.csv')
df_def = pd.read_csv('df_def_cleaned.csv')
df_mid = pd.read_csv('df_mid_cleaned.csv')
df_fwd = pd.read_csv('df_fwd_cleaned.csv')

# Set some initial parameters for the FPL model
budget_limit = 100  # Total budget limit in £m
formation = {'GK': 1, 'DEF': 3, 'MID': 4, 'FWD': 3}  # Initial formation: 1-3-4-3
max_from_team = 3  # Max players allowed from one team

In [42]:
import pandas as pd

# Load the GW 153 predictions data
gw_153_data = pd.read_csv('gw_153_predictions.csv')

# Ensure the data has the necessary columns
print(gw_153_data.columns)


Index(['name', 'position', 'team', 'next_GW_total_points', 'GW', 'season',
       'xP', 'bps', 'goals_conceded', 'kickoff_time', 'opponent_team', 'saves',
       'selected', 'transfers_balance', 'value', 'was_home',
       'expected_goals_conceded', 'fixture_difficulty',
       'rolling_avg_total_points_10gw', 'rolling_avg_clean_sheets_10gw',
       'rolling_avg_saves_10gw', 'rolling_avg_penalties_saved_10gw',
       'Predicted_Points', 'Scorer_Type', 'ict_index', 'expected_assists',
       'expected_goals', 'starts', 'rolling_avg_goals_conceded_10gw',
       'clean_sheets', 'rolling_avg_goals_scored_10gw',
       'rolling_avg_assists_10gw', 'penalties_missed', 'yellow_cards'],
      dtype='object')
